#Building an image retrieval system with deep features


#Fire up GraphLab Create

In [ ]:
import graphlab

#Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [ ]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

#Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [ ]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [ ]:
image_train.head()

#Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [ ]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

#Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [ ]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [ ]:
knn_model.query(cat)

We are going to create a simple function to view the nearest neighbors to save typing:

In [ ]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')


def get_images_from_ids_by_category(query_result, category):
    return category.filter_by(query_result['reference_label'],'id')

In [ ]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

In [ ]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

##Finding similar images to a car

In [ ]:
car = image_train[8:9]
car['image'].show()

In [ ]:
get_images_from_ids(knn_model.query(car))['image'].show()

#Just for fun, let's create a lambda to find and show nearest neighbor images

In [ ]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [ ]:
show_neighbors(8)

In [ ]:
show_neighbors(26)

# 1. sketch_summary of 'label' column

In [ ]:
image_train['label'].sketch_summary()

# 2. Creating category-specific image retrieval models

In [ ]:
dogs = image_train[image_train['label'] == 'dog']
cats = image_train[image_train['label'] == 'cat']
automobiles = image_train[image_train['label'] == 'automobile']
birds = image_train[image_train['label'] == 'bird']

In [ ]:
dog_model = graphlab.nearest_neighbors.create(dogs,features=['deep_features'],
                                             label='id')
cat_model = graphlab.nearest_neighbors.create(cats,features=['deep_features'],
                                             label='id')
automobile_model = graphlab.nearest_neighbors.create(automobiles,features=['deep_features'],
                                             label='id')
bird_model = graphlab.nearest_neighbors.create(birds,features=['deep_features'],
                                             label='id')

In [ ]:
image_test[0:1]['image'].show()

## What is the nearest ‘cat’ labeled image in the training data to the cat image above (the first image in the test data)?

In [ ]:
cat_model.query(image_test[0:1])

## What is the nearest ‘dog’ labeled image in the training data to the cat image above (the first image in the test data)?

In [ ]:
dog_model.query(image_test[0:1])

Experimentando más....

In [ ]:
show_dog_neighbors_in_test = lambda i: get_images_from_ids_by_category(dog_model.query(image_test[i:i+1]),dogs)['image'].show()
show_cat_neighbors_in_test = lambda i: get_images_from_ids_by_category(cat_model.query(image_test[i:i+1]),cats)['image'].show()
show_automobile_neighbors_in_test = lambda i: get_images_from_ids_by_category(automobile_model.query(image_test[i:i+1]),automobiles)['image'].show()
show_bird_neighbors_in_test = lambda i: get_images_from_ids_by_category(bird_model.query(image_test[i:i+1]),birds)['image'].show()

#show_dog_neighbors = lambda i: get_images_from_ids(dog_model.query(image_train[i:i+1]))['image'].show()
#show_cat_neighbors = lambda i: get_images_from_ids(cat_model.query(image_train[i:i+1]))['image'].show()
#show_automobile_neighbors = lambda i: get_images_from_ids(automobile_model.query(image_train[i:i+1]))['image'].show()
#show_bird_neighbors = lambda i: get_images_from_ids(bird_model.query(image_train[i:i+1]))['image'].show()

In [ ]:
show_cat_neighbors_in_test(0)

In [ ]:
show_dog_neighbors_in_test(0)

In [ ]:
get_images_from_ids_by_category(cat_model.query(image_test[0:1]),cats)

## For the first image in the test data , which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled ‘cat’ in the training data

In [ ]:
cat_model.query(image_test[0:1])['distance'].mean()

## For the first image in the test data, which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled ‘dog’ in the training data

In [ ]:
dog_model.query(image_test[0:1])['distance'].mean()

# 4. Computing nearest neighbors accuracy using SFrame operations

In [ ]:
image_test_dog = image_test[image_test['label'] == 'dog']
image_test_cat = image_test[image_test['label'] == 'cat']
image_test_automobile = image_test[image_test['label'] == 'automobile']
image_test_bird = image_test[image_test['label'] == 'bird']

In [ ]:
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)
